## 일상 대화 데이터 전처리 작업

데이터 출처
https://aihub.or.kr/aidata/85

### 대화 발생 장소별 구분

In [1]:
!pwd

/tf/notebooks/NLP


In [2]:
trainDataPath = './train_data'

dataFileList = !(ls $trainDataPath | grep .xlsx)
dataFileList

['A 음식점(15,726)_new.xlsx',
 'B 의류(15,826)_new.xlsx',
 'C 학원(4,773)_new.xlsx',
 'D 소매점(14,949)_new.xlsx',
 'E 생활서비스(11,087)_new.xlsx',
 'F 카페(7,859)_new.xlsx',
 'G 숙박업(7,113)_new.xlsx',
 'H 관광여가오락(4,949)_new.xlsx',
 'I 부동산(8,131)_new.xlsx']

In [143]:
import pandas as pd 
import os
from hanspell import spell_checker
import requests
from urllib import parse
import json
from multiprocessing import Process, Queue
import math
import re
from time import process_time

In [4]:
df = pd.read_excel("./train_data/A 음식점(15,726)_new.xlsx", engine = "openpyxl")

In [5]:
df.head(3)

,SPEAKER,SENTENCE,DOMAINID,DOMAIN,CATEGORY,SPEAKERID,SENTENCEID,MAIN,SUB,QA,QACNCT,MQ,SQ,UA,SA,개체명,용어사전,지식베이스,Unnamed: 18
0,고객,지금 배달되나요?,A,음식점,배달음식점,1,1,배달가능문의,NaN,Q,NaN,지금 배달되나요?,NaN,NaN,NaN,"수고, 지금, 배달",NaN,NaN,NaN
1,점원,아 네 배달됩니다,A,음식점,배달음식점,0,2,배달가능문의,NaN,A,NaN,NaN,NaN,NaN,아 네 배달됩니다,배달,NaN,NaN,NaN
2,고객,짬뽕류는 어떤 게 있나요? 잘 나가는 짬뽕 있나요?,A,음식점,배달음식점,1,3,베스트메뉴문의추천요청,NaN,Q,NaN,짬뽕류는 어떤 게 있나요? 잘 나가는 짬뽕 있나요?,NaN,NaN,NaN,짬뽕,NaN,짬뽕/메뉴,NaN


In [6]:
dataFrames = []
for dataFileName in dataFileList:
    dataFrames.append(pd.read_excel(os.path.join(trainDataPath, dataFileName), engine = "openpyxl"))

In [7]:
dataFrames[0].head(2)

,SPEAKER,SENTENCE,DOMAINID,DOMAIN,CATEGORY,SPEAKERID,SENTENCEID,MAIN,SUB,QA,QACNCT,MQ,SQ,UA,SA,개체명,용어사전,지식베이스,Unnamed: 18
0,고객,지금 배달되나요?,A,음식점,배달음식점,1,1,배달가능문의,NaN,Q,NaN,지금 배달되나요?,NaN,NaN,NaN,"수고, 지금, 배달",NaN,NaN,NaN
1,점원,아 네 배달됩니다,A,음식점,배달음식점,0,2,배달가능문의,NaN,A,NaN,NaN,NaN,NaN,아 네 배달됩니다,배달,NaN,NaN,NaN


In [8]:
dataFrames[-1].head(2)

,SPEAKER,SENTENCE,DOMAINID,DOMAIN,CATEGORY,SPEAKERID,SENTENCEID,MAIN,SUB,QA,QACNCT,MQ,SQ,UA,SA,개체명,용어사전,지식베이스
0,고객,감귤밭 팔려고 하는데 어느정도에 팔 수 있을까요?,I,부동산,토지,1,1,시세문의,NaN,Q,NaN,감귤밭 팔려고 하는데 어느 정도에 팔 수 있을까요?,NaN,NaN,NaN,감귤밭,NaN,감귤밭/매물
1,점원,그쪽 지역은 아직 땅값이 별로 안나가서 많이는 못받을 것 같습니다.,I,부동산,토지,0,2,시세문의,NaN,A,NaN,NaN,NaN,NaN,그쪽 지역은 아직 땅값이 별로 안 나가서 많이는 못 받을 것 같습니다.,"그쪽 지역, 땅값",NaN,그쪽 지역/위치


In [9]:
dataFrames[0]['SENTENCE'][0]

'지금 배달되나요?'

In [10]:
splitedDataFrame = dataFrames[0][['SENTENCE', 'DOMAINID']]
splitedDataFrame.head(2)

,SENTENCE,DOMAINID
0,지금 배달되나요?,A
1,아 네 배달됩니다,A


In [11]:
splitedDataFrame = pd.DataFrame(columns=['SENTENCE', 'DOMAINID'])
splitedDataFrame

,SENTENCE,DOMAINID


In [12]:
splitedDataFrame = pd.DataFrame(columns=['SENTENCE', 'DOMAINID'])
for dfIdx in range(len(dataFrames)):
    sentence_domainid = dataFrames[dfIdx][['SENTENCE', 'DOMAINID']]
    splitedDataFrame = splitedDataFrame.append(sentence_domainid, ignore_index=True)
    
    splitSize = splitedDataFrame['SENTENCE'].size
    sentenceSize = sentence_domainid['SENTENCE'].size
    
    print(f'#{dfIdx}: acc:{splitSize}, len: {sentenceSize}')

#0: acc:15726, len: 15726
#1: acc:31552, len: 15826
#2: acc:36325, len: 4773
#3: acc:51274, len: 14949
#4: acc:62361, len: 11087
#5: acc:70220, len: 7859
#6: acc:77333, len: 7113
#7: acc:82282, len: 4949
#8: acc:90413, len: 8131


In [13]:
splitedDataFrame['SENTENCE'].size

90413

In [14]:
splitedDataFrame.tail(2)

,SENTENCE,DOMAINID
90411,지금 가장 싼 토지 매물 한 번 보여주실래요?,I
90412,평당 90만으로 괜찮은 곳 있어요?,I


In [15]:
splitedDataFrame.dtypes

SENTENCE    object
DOMAINID    object
dtype: object

- 틀린 라벨 수정작업

In [16]:
splitedDataFrame['DOMAINID'].value_counts()

B     15826
A     15726
D     14949
E     11087
I      8131
G      7113
 F     6454
H      4949
C      4773
F      1405
Name: DOMAINID, dtype: int64

In [17]:
splitedDataFrame['DOMAINID'].drop_duplicates().values

array(['A', 'B', 'C', 'D', 'E', 'F', ' F', 'G', 'H', 'I'], dtype=object)

In [18]:
splitedDataFrame['DOMAINID'] = splitedDataFrame['DOMAINID'].replace(
    splitedDataFrame['DOMAINID'].drop_duplicates().values[6], 
    splitedDataFrame['DOMAINID'].drop_duplicates().values[5])

In [19]:
splitedDataFrame['DOMAINID'].value_counts()

B    15826
A    15726
D    14949
E    11087
I     8131
F     7859
G     7113
H     4949
C     4773
Name: DOMAINID, dtype: int64

- 라벨 one hot encoding 작업

In [20]:
splitedDataFrame = pd.get_dummies(splitedDataFrame, columns=["DOMAINID"])
splitedDataFrame.head(2)

,SENTENCE,DOMAINID_A,DOMAINID_B,DOMAINID_C,DOMAINID_D,DOMAINID_E,DOMAINID_F,DOMAINID_G,DOMAINID_H,DOMAINID_I
0,지금 배달되나요?,1,0,0,0,0,0,0,0,0
1,아 네 배달됩니다,1,0,0,0,0,0,0,0,0


In [21]:
splitedDataFrame.tail(2)

,SENTENCE,DOMAINID_A,DOMAINID_B,DOMAINID_C,DOMAINID_D,DOMAINID_E,DOMAINID_F,DOMAINID_G,DOMAINID_H,DOMAINID_I
90411,지금 가장 싼 토지 매물 한 번 보여주실래요?,0,0,0,0,0,0,0,0,1
90412,평당 90만으로 괜찮은 곳 있어요?,0,0,0,0,0,0,0,0,1


In [22]:
splitedDataFrame[35000:35003]

,SENTENCE,DOMAINID_A,DOMAINID_B,DOMAINID_C,DOMAINID_D,DOMAINID_E,DOMAINID_F,DOMAINID_G,DOMAINID_H,DOMAINID_I
35000,성인도 수강 되는건가요?,0,0,1,0,0,0,0,0,0
35001,네 들어오세요,0,0,1,0,0,0,0,0,0
35002,일주일에 며칠 수강하나요?,0,0,1,0,0,0,0,0,0


- 입력 문장 오타 수정 작업

맞춤법에 맞게 수정함으로서 정확도를 높일 수 있지 않을까 하는 생각에 진행

In [23]:
spell_checker.check('안녕하세요.저는한국인입니다.이문장은한글로작성됬습니다.')

Checked(result=True, original='안녕하세요.저는한국인입니다.이문장은한글로작성됬습니다.', checked='안녕하세요. 저는 한국인입니다. 이 문장은 한글로 작성됐습니다.', errors=1, words=OrderedDict([('안녕하세요.', 1), ('저는', 1), ('한국인입니다.', 1), ('이', 1), ('문장은', 1), ('한글로', 1), ('작성됐습니다.', 1)]), time=0.12750744819641113)

In [24]:
spell_checker.check('안녕하세요.저는한국인입니다.이문장은한글로작성됬습니다.').checked

'안녕하세요. 저는 한국인입니다. 이 문장은 한글로 작성됐습니다.'

In [30]:
# spellCheckedList = spell_checker.check(list(splitedDataFrame['SENTENCE']))

너무 오래 걸림

## 네이버 맞춤법 검사기 이용 로직을 직접 짜기

In [43]:
test_sentence = '안녕하세요.저는한국인입니다.이문장은한글로작성됬습니다.'
URL = f'https://m.search.naver.com/p/csearch/ocontent/util/SpellerProxy?q={test_sentence}&where=nexearch&color_blindness=1'
reqQuery = parse.urlencode(parse.parse_qs(parse.urlparse(URL).query))

In [44]:
#f'https://m.search.naver.com/p/csearch/ocontent/util/SpellerProxy?{reqQuery}'
response = requests.get(URL)
response.status_code,response.text

(200,
 '{"message":{"result":{"errata_count":1,"origin_html":"<span class=\'result_underline\'>안녕하세요.저는한국인입니다.이문장은한글로작성됬습니다.</span>","html":"<em class=\'grammar\'>안녕하세요. 저는 한국인입니다. 이 문장은 한글로 작성됐습니다.</em>","notag_html":"안녕하세요. 저는 한국인입니다. 이 문장은 한글로 작성됐습니다."}}}')

In [100]:
processQueue = Queue()
processQueue.qsize()

0

In [138]:
def clean_text(inputString):
    text_rmv = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·\s]', '', inputString)
    return text_rmv

In [124]:
def naver_spell_checker(idx, sentence, use_clean_sentence = True, queue = None):
    URL = f'https://m.search.naver.com/p/csearch/ocontent/util/SpellerProxy?q={clean_text(sentence) if use_clean_sentence else sentence}&where=nexearch&color_blindness=1'
    response = requests.get(URL)
    if response.status_code == 200:
        data = json.loads(response.text)
        if 'message' in data and \
            'result' in data['message'] and \
            'notag_html' in data['message']['result'] and \
            data['message']['result']['notag_html'] is not None:
            if queue:
                queue.put((idx, sentence, data['message']['result']['notag_html']))
            return data['message']['result']['notag_html']
        else:
            print(f'Response error detected. #{idx} sentence: {sentence}')
            print(f'#{idx} response text:', response.text)
            if queue:
                queue.put((idx, sentence, None))
            return None
    else:
        print(f'HTTP Code {response.status_code} != 200, #{idx} sentence: {sentence}')
        if queue:
            queue.put((idx, sentence, None))
        return None

In [139]:
naver_spell_checker(0, '안녕하세요.저는한국인입니다.이문장은한글로작성됬습니다.')

'안녕하세요 저는 한국인입니다 이 문장은 한글로 작성됐습니다'

In [103]:
reqNum = 10
dataSize = 101 #splitedDataFrame['SENTENCE'].size

for pid in range(math.ceil(dataSize / reqNum)):
    start = pid * reqNum + 1
    if start + reqNum > dataSize:
        end = dataSize + 1
    else:
        end = start + reqNum
    processList = []
    for idx in range(start, end):
        print(f'{idx:>4d}', end=' ')
    print()
#     proc = Process(target=naver_spell_checker, args=(splitedDataFrame['SENTENCE'][pid]))
#     proc.start()
#     processList.append(proc)

   1    2    3    4    5    6    7    8    9   10 
  11   12   13   14   15   16   17   18   19   20 
  21   22   23   24   25   26   27   28   29   30 
  31   32   33   34   35   36   37   38   39   40 
  41   42   43   44   45   46   47   48   49   50 
  51   52   53   54   55   56   57   58   59   60 
  61   62   63   64   65   66   67   68   69   70 
  71   72   73   74   75   76   77   78   79   80 
  81   82   83   84   85   86   87   88   89   90 
  91   92   93   94   95   96   97   98   99  100 
 101 


## Process 사용하여 멀티프로세싱 작업

HTTP 통신이 필요한 작업이므로 좀 더 빠른 처리를 위해 동시 요청을 보내 교정작업이 이루어질 수 있도록 구현

### 만약 싱글 프로세싱 방식이었다면?

In [144]:
t1_start = process_time()
for idx in range(21):
    print(naver_spell_checker(idx, splitedDataFrame['SENTENCE'][idx]))
t1_stop = process_time()
print("Elapsed time during the whole program in seconds:", t1_stop - t1_start)

지금 배달되나요
아 네 배달됩니다
짬뽕류는 어떤 게 있나요 잘나가는 짬뽕 있나요
특해물짬뽕도있고전복새우짬뽕도있고해물종류도새우홍합전복없는게없습니다
전복들어가는거는특해물짬뽕시켜야돼요
전복 짬뽕 시키면 전복이 들어가죠
전복 들어가고 여러 가지 또 딴 것도 들어가죠
네네
마찰이 짬뽕밥은 돼지고기 들어가나요
짬뽕은 돼지고기 약간씩 들어갑니다
여기 주소인데 배달되나요
주소는 안됩니다
중국집 명성누죠 배달 지금 가능한가요
예 배달 가능합니다
주로 어떤 게 잘나가요
탕수육에 짜장이나 짬뽕 세트가 잘나가죠
4인이 먹을 수 있을까요
네 명 드시기엔 세트가 양이 적어요 네 명 드시려면 탕수육 중자는 가야죠
중자는 얼마예요
중자는 2만 2천 원요
탕수육에 들어가는 고기가 돼지고기죠
Elapsed time during the whole program in seconds: 0.2715468920000035


### 만약 멀티 프로세싱 방식이라면?

In [145]:
while not processQueue.empty():
    try:
        processQueue.get(False)
    except Empty:
        continue
#     processQueue.task_done()
processQueue.qsize()

0

In [146]:
reqNum = 10
dataSize = 21 #splitedDataFrame['SENTENCE'].size
t1_start = process_time()

for pid in range(math.ceil(dataSize / reqNum)):
    start = pid * reqNum + 1
    if start + reqNum > dataSize:
        end = dataSize + 1
    else:
        end = start + reqNum
    processList = []
    for idx in range(start, end):
        proc = Process(target=naver_spell_checker, args=(idx, splitedDataFrame['SENTENCE'][idx], False, processQueue))
        proc.start()
        processList.append(proc)
    for proc in processList:
        proc.join()
t1_stop = process_time()

In [147]:
while not processQueue.empty():
    print(processQueue.get(0))

(1, '아 네 배달됩니다', '아 네 배달됩니다')
(2, '짬뽕류는 어떤 게 있나요? 잘 나가는 짬뽕 있나요?', '짬뽕류는 어떤 게 있나요? 잘나가는 짬뽕 있나요?')
(5, '전복 짬뽕 시키면 전복이 들어가죠', '전복 짬뽕 시키면 전복이 들어가죠')
(3, '특해물 짬뽕도 있고 전복 새우 짬뽕도 있고 해물 종류도 새우 홍합 전복 없는 게 없습니다', '특해물 짬뽕도 있고 전복 새우 짬뽕도 있고 해물 종류도 새우 홍합 전복 없는 게 없습니다')
(4, '전복 들어가는 거는 특해물 짬뽕 시켜야 돼요?', '전복 들어가는 거는 특해물 짬뽕 시켜야 돼요?')
(7, '네네', '네네')
(6, '전복 들어가고 여러 가지 또 딴 것도 들어가죠?', '전복 들어가고 여러 가지 또 딴 것도 들어가죠?')
(8, '마찰이 짬뽕밥은 돼지고기 들어가나요?', '마찰이 짬뽕밥은 돼지고기 들어가나요?')
(10, '여기 #주소#인데 배달되나요?', '여기 ')
(9, '짬뽕은 돼지고기 약간씩 들어갑니다', '짬뽕은 돼지고기 약간씩 들어갑니다')
(11, '#주소#는 안됩니다', '')
(13, '예 배달 가능합니다', '예 배달 가능합니다')
(12, '중국집 명성 누죠? 배달 지금 가능한가요?', '중국집 명성 누죠? 배달 지금 가능한가요?')
(14, '주로 어떤 게 잘 나가요?', '주로 어떤 게 잘나가요?')
(15, '탕수육에 짜장이나 짬뽕 세트가 잘 나가죠 ', '탕수육에 짜장이나 짬뽕 세트가 잘나가죠 ')
(16, '4인이 먹을 수 있을까요?', '4인이 먹을 수 있을까요?')
(18, '중자는 얼마에요?', '중자는 얼마예요?')
(17, '네 명 드시기엔 세트가 양이 적어요 네 명 드시려면 탕수육 중자는 가야죠', '네 명 드시기엔 세트가 양이 적어요 네 명 드시려면 탕수육 중자는 가야죠')
(19, '중자는 2만 2천 원요', '중자는 2만 2천 원요')
(20, '탕수육에 들어가는 고기가 돼지고기죠?', '탕수육에 들어가는 고기가 돼지고기죠?')
(21

In [148]:
print("Elapsed time during the whole program in seconds:", t1_stop - t1_start)

Elapsed time during the whole program in seconds: 0.15046933200000012


- 싱글 프로세싱 시간: `0.27`s
- 멀티 프로세싱 시간: `0.15`s

해당 시간은 `21`건을 기준으로 하였으므로 현재 가지고 있는 데이터 `90413`건으로 예측한다면

- 싱글 프로세싱 시간: `1162.45`s 약 19분
- 멀티 프로세싱 시간: `645.80`s 약 10분

으로 차이가 극명히 나기 시작함

대신 정렬이 되어지지 않기 때문에 정렬 후처리가 필요하다는 단점 존재

__위 내용에서 보면 `#주소#` 와 같이 특수문자가 들어간 내용의 맞춤법 검사 결과는 기존 문장을 삭제한 결과가 나오는 것을 확인 할 수 있음__

특수문자에 대한 추가적인 전처리가 필요

In [140]:
while not processQueue.empty():
    try:
        processQueue.get(False)
    except Empty:
        continue
#     processQueue.task_done()
processQueue.qsize()

0

In [141]:
reqNum = 10
dataSize = 21 #splitedDataFrame['SENTENCE'].size

for pid in range(math.ceil(dataSize / reqNum)):
    start = pid * reqNum + 1
    if start + reqNum > dataSize:
        end = dataSize + 1
    else:
        end = start + reqNum
    processList = []
    for idx in range(start, end):
        proc = Process(target=naver_spell_checker, args=(idx, splitedDataFrame['SENTENCE'][idx], True, processQueue))
        proc.start()
        processList.append(proc)
    for proc in processList:
        proc.join()

In [142]:
while not processQueue.empty():
    print(processQueue.get(0))

(1, '아 네 배달됩니다', '아 네 배달됩니다')
(2, '짬뽕류는 어떤 게 있나요? 잘 나가는 짬뽕 있나요?', '짬뽕류는 어떤 게 있나요 잘나가는 짬뽕 있나요')
(7, '네네', '네네')
(4, '전복 들어가는 거는 특해물 짬뽕 시켜야 돼요?', '전복들어가는거는특해물짬뽕시켜야돼요')
(5, '전복 짬뽕 시키면 전복이 들어가죠', '전복 짬뽕 시키면 전복이 들어가죠')
(9, '짬뽕은 돼지고기 약간씩 들어갑니다', '짬뽕은 돼지고기 약간씩 들어갑니다')
(6, '전복 들어가고 여러 가지 또 딴 것도 들어가죠?', '전복 들어가고 여러 가지 또 딴 것도 들어가죠')
(8, '마찰이 짬뽕밥은 돼지고기 들어가나요?', '마찰이 짬뽕밥은 돼지고기 들어가나요')
(10, '여기 #주소#인데 배달되나요?', '여기 주소인데 배달되나요')
(3, '특해물 짬뽕도 있고 전복 새우 짬뽕도 있고 해물 종류도 새우 홍합 전복 없는 게 없습니다', '특해물짬뽕도있고전복새우짬뽕도있고해물종류도새우홍합전복없는게없습니다')
(11, '#주소#는 안됩니다', '주소는 안됩니다')
(13, '예 배달 가능합니다', '예 배달 가능합니다')
(12, '중국집 명성 누죠? 배달 지금 가능한가요?', '중국집 명성누죠 배달 지금 가능한가요')
(14, '주로 어떤 게 잘 나가요?', '주로 어떤 게 잘나가요')
(15, '탕수육에 짜장이나 짬뽕 세트가 잘 나가죠 ', '탕수육에 짜장이나 짬뽕 세트가 잘나가죠')
(16, '4인이 먹을 수 있을까요?', '4인이 먹을 수 있을까요')
(17, '네 명 드시기엔 세트가 양이 적어요 네 명 드시려면 탕수육 중자는 가야죠', '네 명 드시기엔 세트가 양이 적어요 네 명 드시려면 탕수육 중자는 가야죠')
(19, '중자는 2만 2천 원요', '중자는 2만 2천 원요')
(18, '중자는 얼마에요?', '중자는 얼마예요')
(20, '탕수육에 들어가는 고기가 돼지고기죠?', '탕수육에 들어가는 고기가 돼지고기죠')
(21, '어느 집이든지 다 돼지

이상하게도 `잘 나가는` 용어가 맞춤법 검사를 하면 `잘나가는` 으로 수정하는 문제가 존재

https://www.korean.go.kr/front/onlineQna/onlineQnaView.do?mn_id=216&qna_seq=112284

국립국어원에선 `잘 나가는`을 적절한 용어라고 말하고 있음

또한 특문 제거 로직에서 스페이스도 삭제하도록 하였더니 일부 문장에선 아예 맞춤법 검사가 실패한 모습을 볼 수 있음

`특해물짬뽕도있고전복새우짬뽕도있고해물종류도새우홍합전복없는게없습니다`

한국어 토큰 분리를 이용하여 해당 부분을 개선해보자!

## 한국어 토큰 분리 모듈 사용